# Local Experiment Tracking (No External Services)

## Objective
This notebook demonstrates how to track deep learning experiments locally

We log:
- Hyperparameters
- Training and validation metrics
- Model artifacts

We are logging this for later comparison. This mirrors what tools like MLflow or W&B automate.

(W&B unresolved issues)

In [49]:
import torch
import torch.nn as nn
import torch.optim as optim

import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader

import csv
import os


In [50]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

Using device: cpu


In [51]:
BASE_CONFIG = {
    "batch_size": 64,
    "epochs": 5,
    "num_workers": 2
}


In [52]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5),
                         (0.5, 0.5, 0.5))
])


In [55]:
train_dataset = torchvision.datasets.CIFAR10(
    root="./data",
    train=True,
    download=True,
    transform=transform
)

test_dataset = torchvision.datasets.CIFAR10(
    root="./data",
    train=False,
    download=True,
    transform=transform
)

train_dataset = torch.utils.data.Subset(train_dataset, range(5000))
test_dataset = torch.utils.data.Subset(test_dataset, range(1000))

Files already downloaded and verified
Files already downloaded and verified


In [56]:
train_loader = DataLoader(
    train_dataset,
    batch_size=BASE_CONFIG["batch_size"],
    shuffle=True,
    num_workers=BASE_CONFIG["num_workers"]
)

test_loader = DataLoader(
    test_dataset,
    batch_size=BASE_CONFIG["batch_size"],
    shuffle=False,
    num_workers=BASE_CONFIG["num_workers"]
)


In [57]:
class SimpleCNN(nn.Module):
    def __init__(self):
        super().__init__()

        self.features = nn.Sequential(
            nn.Conv2d(3, 32, 3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),

            nn.Conv2d(32, 64, 3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )

        self.classifier = nn.Sequential(
            nn.Linear(64 * 8 * 8, 128),
            nn.ReLU(),
            nn.Linear(128, 10)
        )

    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size(0), -1)
        return self.classifier(x)


In [58]:
def train_epoch(model, loader, criterion, optimizer, device):
    model.train()
    total_loss = 0
    correct = 0
    total = 0

    for images, labels in loader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        _, preds = torch.max(outputs, 1)
        correct += (preds == labels).sum().item()
        total += labels.size(0)

    return total_loss / len(loader), correct / total


In [59]:
def evaluate(model, loader, criterion, device):
    model.eval()
    total_loss = 0
    correct = 0
    total = 0

    with torch.no_grad():
        for images, labels in loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)

            total_loss += loss.item()
            _, preds = torch.max(outputs, 1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)

    return total_loss / len(loader), correct / total


In [60]:
def run_experiment(config, run_name):
    model = SimpleCNN().to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=config["lr"])

    results = []

    for epoch in range(config["epochs"]):
        train_loss, train_acc = train_epoch(
            model, train_loader, criterion, optimizer, device
        )
        val_loss, val_acc = evaluate(
            model, test_loader, criterion, device
        )

        results.append([epoch, train_loss, train_acc, val_acc])

        print(
            f"{run_name} | Epoch {epoch+1} | "
            f"Train Acc: {train_acc:.4f} | Val Acc: {val_acc:.4f}"
        )

    return results


In [61]:
experiments = [
    {"lr": 1e-3, "epochs": 5},
    {"lr": 1e-4, "epochs": 5}
]

In [62]:
os.makedirs("logs", exist_ok=True)

for i, exp in enumerate(experiments):
    config = {**BASE_CONFIG, **exp}
    results = run_experiment(config, run_name=f"run_{i}")

    with open(f"logs/run_{i}.csv", "w", newline="") as f:
        writer = csv.writer(f)
        writer.writerow(["epoch", "train_loss", "train_acc", "val_acc"])
        writer.writerows(results)

run_0 | Epoch 1 | Train Acc: 0.2996 | Val Acc: 0.3810
run_0 | Epoch 2 | Train Acc: 0.4432 | Val Acc: 0.4610
run_0 | Epoch 3 | Train Acc: 0.5032 | Val Acc: 0.4610
run_0 | Epoch 4 | Train Acc: 0.5562 | Val Acc: 0.5070
run_0 | Epoch 5 | Train Acc: 0.6020 | Val Acc: 0.5550
run_1 | Epoch 1 | Train Acc: 0.2372 | Val Acc: 0.2870
run_1 | Epoch 2 | Train Acc: 0.3230 | Val Acc: 0.3680
run_1 | Epoch 3 | Train Acc: 0.3726 | Val Acc: 0.3710
run_1 | Epoch 4 | Train Acc: 0.4010 | Val Acc: 0.4150
run_1 | Epoch 5 | Train Acc: 0.4326 | Val Acc: 0.4360
